# 벡터 : chroma vs pinecone
- chroma : 인메모리 vector DB, 로컬메모리 vector DB
- pinecone : 클라우드 vector DB
    * pinecone consoledp API key 생성 -> .env(PINECONE_API_KEY)


# 0. 패키지 설치

In [ ]:
!pip install pinecone-client pinecone-text

# 1. Knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

document_list = loader.load_and_split(text_splitter=text_splitter)

len(document_list)

183

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
index_name = 'tax-index-upstage'


In [10]:
# 데이터를 처음 업로드할 때
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)

In [9]:
# 데이터를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,
    index_name=index_name
)

In [11]:
database

# 2. 답변 생성을 위한 Retrieval 

In [12]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.as_retriever(search_kwargs={"k": 3})
retriever.invoke(query)

[Document(id='cda3359f-8ddd-47df-a9b0-230dd8c4e321', metadata={'source': 'tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것

# 3. 제공되는 prompt를 활용하여 답변 생성

In [13]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt')

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

In [15]:
message = qa_chain.invoke({"query":query})
print(message['result'])

연봉 5천만원인 직장인의 소득세는 정확히 계산하기 어렵지만, 근로소득공제와 자녀세액공제 등을 고려하면 대략 300만 원에서 400만 원 사이일 수 있습니다. 이 금액은 근로소득공제, 자녀세액공제, 그리고 기타 공제 항목에 따라 달라질 수 있습니다. 구체적인 세액 계산을 위해서는 상세한 소득 및 공제액 정보를 확인해야 합니다.


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
dictionary = ['사람을 나타내는 표현 -> 거주자']
prompt = ChatPromptTemplate.from_template(f""" 사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의
                                          질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
                                          그런 경우에는 질문만 리턴해 주세요.
                                          사전 : {dictionary}
                                          사용자 질문 : {{query}}""")